In [1]:
# import os
# import sys
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import numpy as np
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.evaluators.evaluators import eval_is_below, combine_evals, eval_is_above
from quantfreedom.plotting.plot_helper_functions import *
from quantfreedom.plotting.strat_dash import *
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)

prices = pd.read_csv(
    "data/30min.csv", index_col="time"
)

In [2]:
rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[15,30,60],
)
rsi_eval = eval_is_below(
    want_to_evaluate = rsi_ind,
    user_args=np.arange(30, 51, 10),
)
ema_ind = from_talib(
    func_name='ema',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[200, 400, 600]
)
ema_eval = eval_is_below(
    want_to_evaluate=ema_ind,
    df_prices=prices,
    cand_ohlc='close',
)
entries = combine_evals(
    bigger_df=rsi_eval,
    smaller_df=ema_eval,
)
entry = entries.iloc[:, [5]]
final_array, order_records = simulate_up_to_6_nb(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entry.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=5,
    risk_rewards=10,
    size_pct=1.,
    # sl_pcts=[2,3,4],
    tsl_true_or_false=True,
    tsl_pcts_init=1,
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=2,
    tsl_when_pct_from_avg_entry=3,
)


In [3]:
list(ema_ind[600].columns.names)

['EMA_timeperiod']

In [4]:
entries.loc[:, (slice(15), slice(30), slice(200))]

RSI_timeperiod,15
RSI_is_below,30
EMA_timeperiod,200
EMA_is_below,close_price
time,
2020-10-01 00:00:00,False
2020-10-01 00:30:00,False
2020-10-01 01:00:00,False
2020-10-01 01:30:00,False
2020-10-01 02:00:00,False
...,...


In [5]:
indicator_dict = {
    "candle_chart": {
        "values1": ema_ind[600],
        "entries1": ema_eval[600]['close_price'],
    },
    "indicator1": {
        "values1": rsi_ind[15],
        "entries1": rsi_eval[15][40],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=prices,
    order_records=order_records,
)

Dash is running on http://127.0.0.1:3003/

Dash app running on http://127.0.0.1:3003/
